In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
!pip install xgboost
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight')

  Using cached https://files.pythonhosted.org/packages/4f/4c/4969b10939c4557ae46e5569d07c0c7ce772b3d6b9c1401a6ed07059fdee/xgboost-0.81.tar.gz
    Complete output from command python setup.py egg_info:
    ++ pwd
    + oldpath=/private/var/folders/4p/sglgj7w57gv9jq8wgwltnhfm0000gn/T/pip-install-vrfvand_/xgboost
    + cd ./xgboost/
    + echo darwin18
    + grep -q darwin
    + LIB_XGBOOST=libxgboost.dylib
    + which g++-5
    + which g++-7
    + which clang++
    /anaconda3/envs/PythonData/bin/clang++
    + export CC=clang
    + CC=clang
    + export CXX=clang++
    + CXX=clang++
    + make clean
    xcrun: error: invalid active developer path (/Library/Developer/CommandLineTools), missing xcrun at: /Library/Developer/CommandLineTools/usr/bin/xcrun
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/4p/sglgj7w57gv9jq8wgwltnhfm0000gn/T/pip-install-vrfvand_/xgboost/setup.py", line 42, in <module>
        LIB_PATH = libpath['f

ModuleNotFoundError: No module named 'xgboost'

Data

In [ ]:
energy = pd.read_csv('../Energy_Project/Resources/AEP_hourly.csv', index_col=[0], parse_dates=[0])

In [ ]:
color_pal = ["#F8766D", "#D39200", "#93AA00", "#00BA38", "#00C19F", "#00B9E3", "#619CFF", "#DB72FB"]
_ = energy.plot(style='.', figsize=(15,5), color=color_pal[0], title='AEP')

Train/Test Split

In [ ]:
split_date = '01-Oct-2014'
energy_train = energy.loc[energy.index <= split_date].copy()
energy_test = energy.loc[energy.index > split_date].copy()

In [ ]:
_ = energy_test \
    .rename(columns={'AEP_MW': 'TEST SET'}) \
    .join(energy_train.rename(columns={'AEP_MW': 'TRAINING SET'}), how='outer') \
    .plot(figsize=(15,5), title='AEP', style='.')

Create Time Series Features

In [ ]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
X_train, y_train = create_features(energy_train, label='AEP_MW')
X_test, y_test = create_features(energy_test, label='AEP_MW')

Create XGBoost Model

In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
       verbose=False) # Change verbose to True if you want to see it train

Feature Importances
Feature importance is a great way to get a general idea about which features the model is relying on most to make the prediction. This is a metric that simply sums up how many times each feature is split on.

We can see that the day of year was most commonly used to split trees, while hour and year came in next. Quarter has low importance due to the fact that it could be created by different dayofyear splits.

In [ ]:
_ = plot_importance(reg, height=0.9)

Forecast on Test Set

In [ ]:
energy_test['MW_Prediction'] = reg.predict(X_test)
energy_all = pd.concat([energy_test, energy_train], sort=False)

In [ ]:
_ = energy_all[['AEP_MW','MW_Prediction']].plot(figsize=(15, 5))

Look at the First Month of Predictions

In [ ]:
# Plot the forecast with the actuals
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
_ = energy_all[['MW_Prediction','AEP_MW']].plot(ax=ax,
                                              style=['-','.'])
ax.set_xbound(lower='10-01-2014', upper='11-01-2014')
ax.set_ylim(0, 60000)
plot = plt.suptitle('October 2014 Forecast vs Actuals')

In [ ]:
# Plot the forecast with the actuals
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
_ = energy_all[['MW_Prediction','AEP_MW']].plot(ax=ax,
                                              style=['-','.'])
ax.set_xbound(lower='10-01-2014', upper='10-08-2014')
ax.set_ylim(0, 60000)
plot = plt.suptitle('First Week of October Forecast vs Actuals')

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
_ = energy_all[['MW_Prediction','AEP_MW']].plot(ax=ax,
                                              style=['-','.'])
ax.set_ylim(0, 60000)
ax.set_xbound(lower='07-01-2015', upper='07-08-2015')
plot = plt.suptitle('First Week of July Forecast vs Actuals')

Error Metrics on Test

Our RMSE error is 2636882/
Our MAE error is 1295.63/
Our MAPE error is 8.8%

In [ ]:
mean_squared_error(y_true=energy_test['AEP_MW'],
                   y_pred=energy_test['MW_Prediction'])

In [ ]:
mean_absolute_error(y_true=energy_test['AEP_MW'],
                   y_pred=energy_test['MW_Prediction'])

We are using mean absolute percent error because it gives an easy to interperate percentage showing how off the predictions are. MAPE isn't included in sklearn so we need to use a custom function.

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(y_true=energy_test['AEP_MW'],
                   y_pred=energy_test['MW_Prediction'])

Look at Worst and best Predicted Days

In [ ]:
energy_test['error'] = energy_test['AEP_MW'] - energy_test['MW_Prediction']
energy_test['abs_error'] = energy_test['error'].apply(np.abs)
error_by_day = energy_test.groupby(['year','month','dayofmonth']) \
    .mean()[['AEP_MW','MW_Prediction','error','abs_error']]

In [ ]:
# Over forecasted days
error_by_day.sort_values('error', ascending=True).head(10)

Interesting Findings: The top dates are holidays; it would be cool to chart weather data against this to see if there was a cold front at the end of February

In [ ]:
# Worst absolute predicted days
error_by_day.sort_values('abs_error', ascending=False).head(10)

The best predicted days seem in October

In [ ]:
# Best predicted days
error_by_day.sort_values('abs_error', ascending=True).head(10)

Plotting some best/worst predicted days

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(10)
_ = energy_all[['MW_Prediction','AEP_MW']].plot(ax=ax,
                                              style=['-','.'])
ax.set_ylim(0, 60000)
ax.set_xbound(lower='02-19-2015', upper='02-20-2015')
plot = plt.suptitle('Feb 19, 2015 - Worst Predicted Day')

It was a high of 8 degrees in West Virginia on 02/19/2015, well below the average daily temperature of 34, and a low of 1 degree! In short, it was colder than normal which may have resulted in the model's undestimation.

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(10)
_ = energy_all[['MW_Prediction','AEP_MW']].plot(ax=ax,
                                              style=['-','.'])
ax.set_ylim(0, 60000)
ax.set_xbound(lower='03-13-2015', upper='03-14-2015')
plot = plt.suptitle('March 13, 2015 - Best Predicted Day')

This one is nearly spot on!

Things to investigate: Weather Data, Holiday Indicators, Local Economic Activity Analysis